Import Statements

In [15]:
import cv2
from otsu_helper import *
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

Load the Image Set

In [16]:
imageSet = "miku2"
raw_input_image, grey_input_image = loadImages(imageSet)
raw_input_image.shape

(716, 717, 3)

Perform Image Segmentation Using RGB values

In [17]:
# split the raw input image into 3 individual channels
blueChannel, greenChannel, redChannel = cv2.split(raw_input_image)

# apply erosion then dilation of the image
# kernel = np.ones((5, 5), np.uint8)
# blueChannel_mask = cv2.erode(blueChannel, kernel, iterations=5)
# greenChannel_mask = cv2.erode(greenChannel, kernel, iterations=5)
# redChannel_mask = cv2.erode(redChannel, kernel, iterations=5)
#
# blueChannel_mask = cv2.dilate(blueChannel, kernel, iterations=5)
# greenChannel_mask = cv2.dilate(greenChannel, kernel, iterations=5)
# redChannel_mask = cv2.dilate(redChannel, kernel, iterations=5)

# apply segmentation on each individual channel
blueChannel_mask = minWithinClassVariance(blueChannel)
greenChannel_mask = minWithinClassVariance(greenChannel)
redChannel_mask = minWithinClassVariance(redChannel)

# # combine all masks together


  0%|          | 0/242 [00:00<?, ?it/s]/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 242/242 [00:33<00:00,  7.24it/s]


best threshold:117


100%|██████████| 254/254 [00:35<00:00,  7.25it/s]


best threshold:111


100%|██████████| 256/256 [00:35<00:00,  7.27it/s]

best threshold:119


In [18]:
mask_all = np.ones(blueChannel.shape)
mask_all = np.logical_and(mask_all, blueChannel_mask)
mask_all = np.logical_and(mask_all, redChannel_mask)
mask_all = np.logical_and(mask_all, greenChannel_mask).astype(np.uint8) * 255

all_channels_plot = np.hstack((blueChannel_mask, greenChannel_mask))
all_channels_plot = np.hstack((all_channels_plot, redChannel_mask))
all_channels_plot = np.hstack((all_channels_plot, mask_all))

cv2.imwrite(f'/home/jo_wang/Desktop/ECE661/HW06/rgb_suplots/{imageSet}_rgb_subplot.jpg', all_channels_plot)
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/rgb_segmentation/{imageSet}_rgb_seg.jpg", mask_all)


True

Perform Texture Based Segmentation

In [19]:
ch1 = performTexture(grey_input_image, 3)
ch2 = performTexture(grey_input_image, 5)
ch3 = performTexture(grey_input_image, 7)

ch1_otsu = minWithinClassVariance(ch1)
ch2_otsu = minWithinClassVariance(ch2)
ch3_otsu = minWithinClassVariance(ch3)

texture_all = np.ones(ch1_otsu.shape)
texture_all = np.logical_and(texture_all, ch1_otsu)
texture_all = np.logical_and(texture_all, ch2_otsu)

all_texture_subplot = np.hstack((np.logical_not(ch1_otsu).astype(np.uint8) * 255, np.logical_not(ch2_otsu).astype(np.uint8) * 255))
all_texture_subplot = np.hstack((all_texture_subplot, np.logical_not(ch3_otsu).astype(np.uint8) * 255))
all_texture_subplot = np.hstack((all_texture_subplot, np.logical_not(texture_all).astype(np.uint8) * 255))
cv2.imwrite(f'/home/jo_wang/Desktop/ECE661/HW06/texture_subplot/{imageSet}_texture_subplot.jpg', all_texture_subplot)
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/texture_segmentation/{imageSet}_texture_seg.jpg", np.logical_not(texture_all).astype(np.uint8) * 255)


100%|██████████| 256/256 [00:35<00:00,  7.22it/s]


best threshold:76


100%|██████████| 256/256 [00:34<00:00,  7.50it/s]


best threshold:81


100%|██████████| 256/256 [00:32<00:00,  7.81it/s]

best threshold:83


True

Extract Contours

In [20]:
texture_all = np.logical_not(texture_all).astype(np.uint8) * 255

# kernel = np.ones((5, 5), np.uint8)
# texture_all = cv2.erode(texture_all, kernel, iterations=1)
# texture_all = cv2.dilate(texture_all, kernel, iterations=1)

contourImage = getContour(np.logical_not(texture_all // 255).astype(np.uint8))
contourRGB = getContour(np.logical_not(mask_all // 255).astype(np.uint8))
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/contour_extraction/{imageSet}_contour_rgb.jpg", np.logical_not(contourRGB // 255).astype(np.uint8) * 255)
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/contour_extraction/{imageSet}_contour_texture.jpg", np.logical_not(contourImage // 255).astype(np.uint8) * 255)

True

In [21]:
print(texture_all.shape)
print(contourImage.shape)
# og_bgr = np.hstack((raw_input_image, cv2.cvtColor(mask_all, cv2.COLOR_GRAY2RGB)))
# texture_contour = np.hstack((cv2.cvtColor(texture_all, cv2.COLOR_GRAY2RGB), cv2.cvtColor(np.logical_not(contourImage // 255).astype(np.uint8) * 255, cv2.COLOR_GRAY2RGB) ) )
# final_image = np.vstack((og_bgr, texture_contour))

top = np.hstack((mask_all, np.logical_not(contourRGB // 255).astype(np.uint8) * 255))
bottom = np.hstack((texture_all, np.logical_not(contourImage // 255).astype(np.uint8) * 255))
final = np.vstack((top,bottom))
# rgb seg           rgb contour
# texture seg       texture contour
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/final_subplots/{imageSet}_final_subplots.jpg", final)


(716, 717)
(716, 717)


True